In [ ]:
!pip install ultralytics

import os
import cv2
import numpy as np
from ultralytics import YOLO

In [ ]:
images_dir = 'person_resume_funsd_format_v5/dataset/testing/images'       
output_dir = 'person_resume_funsd_format_v5/dataset/processed_data/labels'         

os.makedirs(output_dir, exist_ok=True)

model = YOLO('models/best.pt')


In [ ]:
def convert_bbox_to_yolo_format(box, img_width, img_height):
    """
    Converts an absolute bbox [xmin, ymin, xmax, ymax] into YOLO format: 
    (x_center, y_center, width, height) as relative coordinates.
    """
    xmin, ymin, xmax, ymax = box
    x_center = (xmin + xmax) / 2.0 / img_width
    y_center = (ymin + ymax) / 2.0 / img_height
    bbox_width = (xmax - xmin) / img_width
    bbox_height = (ymax - ymin) / img_height
    return x_center, y_center, bbox_width, bbox_height


In [ ]:
class_mapping = {
    'Experience': 0,
    'Personal Info': 1,
    'Skills': 2
}

# Loop through each image file in the images directory
for img_name in os.listdir(images_dir):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(images_dir, img_name)
        # Read image using OpenCV
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue

        img_height, img_width = img.shape[:2]
        
        # Run YOLO prediction
        results = model(img_path)[0]  # Get the first result (for the image)
        
        # Initialize list to hold YOLO formatted predictions
        yolo_lines = []
        
        # Loop through each prediction (bounding box)
        for pred in results.boxes.data.cpu().numpy():
            # pred format: [xmin, ymin, xmax, ymax, score, class]
            xmin, ymin, xmax, ymax, score, cls = pred
            
            # Convert the bbox to YOLO format
            x_center, y_center, bbox_width, bbox_height = convert_bbox_to_yolo_format(
                [xmin, ymin, xmax, ymax], img_width, img_height
            )
            
            # Convert class to integer (if needed, based on your training setup)
            class_idx = int(cls)
            # In our mapping, we assume the model was trained with 0,1,2 corresponding to our desired classes.
            # If you have a mapping between model's class names and our desired labels, adjust here.
            
            # Format: class x_center y_center width height
            line = f"{class_idx} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}"
            yolo_lines.append(line)
        
        # Save the YOLO labels to a .txt file with the same name as the image
        txt_filename = os.path.splitext(img_name)[0] + '.txt'
        txt_path = os.path.join(output_dir, txt_filename)
        with open(txt_path, 'w') as f:
            f.write("\n".join(yolo_lines))
        
        print(f"Processed {img_name} and saved labels to {txt_filename}")
